In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

### Data preprocessing

In [43]:
def load_file(path):
    return pd.read_csv(path, delim_whitespace=True, header=None)
X_train=[]
X_train.append(load_file("./UCI HAR dataset/UCI HAR dataset/train/Inertial Signals/body_acc_x_train.txt").to_numpy())
X_train.append(load_file("./UCI HAR dataset/UCI HAR dataset/train/Inertial Signals/body_acc_y_train.txt").to_numpy())
X_train.append(load_file("./UCI HAR dataset/UCI HAR dataset/train/Inertial Signals/body_acc_z_train.txt").to_numpy())
X_train.append(load_file("./UCI HAR dataset/UCI HAR dataset/train/Inertial Signals/body_gyro_x_train.txt").to_numpy())
X_train.append(load_file("./UCI HAR dataset/UCI HAR dataset/train/Inertial Signals/body_gyro_y_train.txt").to_numpy())
X_train.append(load_file("./UCI HAR dataset/UCI HAR dataset/train/Inertial Signals/body_gyro_z_train.txt").to_numpy())
X_train.append(load_file("./UCI HAR dataset/UCI HAR dataset/train/Inertial Signals/total_acc_x_train.txt").to_numpy())
X_train.append(load_file("./UCI HAR dataset/UCI HAR dataset/train/Inertial Signals/total_acc_y_train.txt").to_numpy())
X_train.append(load_file("./UCI HAR dataset/UCI HAR dataset/train/Inertial Signals/total_acc_z_train.txt").to_numpy())
X_train=np.transpose(X_train, (1,2,0))

In [45]:
X_test=[]
X_test.append(load_file("./UCI HAR dataset/UCI HAR dataset/test/Inertial Signals/body_acc_x_test.txt").to_numpy())
X_test.append(load_file("./UCI HAR dataset/UCI HAR dataset/test/Inertial Signals/body_acc_y_test.txt").to_numpy())
X_test.append(load_file("./UCI HAR dataset/UCI HAR dataset/test/Inertial Signals/body_acc_z_test.txt").to_numpy())
X_test.append(load_file("./UCI HAR dataset/UCI HAR dataset/test/Inertial Signals/body_gyro_x_test.txt").to_numpy())
X_test.append(load_file("./UCI HAR dataset/UCI HAR dataset/test/Inertial Signals/body_gyro_y_test.txt").to_numpy())
X_test.append(load_file("./UCI HAR dataset/UCI HAR dataset/test/Inertial Signals/body_gyro_z_test.txt").to_numpy())
X_test.append(load_file("./UCI HAR dataset/UCI HAR dataset/test/Inertial Signals/total_acc_x_test.txt").to_numpy())
X_test.append(load_file("./UCI HAR dataset/UCI HAR dataset/test/Inertial Signals/total_acc_y_test.txt").to_numpy())
X_test.append(load_file("./UCI HAR dataset/UCI HAR dataset/test/Inertial Signals/total_acc_z_test.txt").to_numpy())
X_test=np.transpose(X_test, (1,2,0))

In [2]:
y_train=pd.read_csv("./UCI HAR dataset/UCI HAR dataset/train/y_train.txt", delim_whitespace=True, header=None)
y_test=pd.read_csv("./UCI HAR dataset/UCI HAR dataset/test/y_test.txt", delim_whitespace=True, header=None)

In [44]:
np.shape(X_train)

(7352, 128, 9)

In [8]:
activity_labels=pd.read_csv('./UCI HAR dataset/UCI HAR dataset/activity_labels.txt', delim_whitespace=True, header=None)

In [9]:
dict(activity_labels.values)

{1: 'WALKING',
 2: 'WALKING_UPSTAIRS',
 3: 'WALKING_DOWNSTAIRS',
 4: 'SITTING',
 5: 'STANDING',
 6: 'LAYING'}

In [10]:
y_train[0]=y_train[0].map(dict(activity_labels.values))
y_test[0]=y_test[0].map(dict(activity_labels.values))
y_train.columns=['Activity']
y_test.columns=['Activity']

In [11]:
y_train

,Activity
0,STANDING
1,STANDING
2,STANDING
3,STANDING
4,STANDING
...,...
7347,WALKING_UPSTAIRS
7348,WALKING_UPSTAIRS
7349,WALKING_UPSTAIRS
7350,WALKING_UPSTAIRS


In [16]:
encoder=OneHotEncoder(sparse_output=False)

In [17]:
y_train=encoder.fit_transform(y_train)
y_test=encoder.fit_transform(y_test)

In [46]:
np.shape(X_train)

(7352, 128, 9)

In [47]:
np.shape(X_test)

(2947, 128, 9)

### Building Model

In [48]:
n_timesteps, n_features, n_outputs=X_train.shape[1], X_train.shape[2], y_train.shape[1]

In [49]:
n_outputs

6

In [50]:
n_timesteps

128

In [51]:
n_features

9

In [52]:
conv1d_model=tf.keras.Sequential()
conv1d_model.add(tf.keras.layers.Conv1D(filters=70, kernel_size=3, activation='relu', input_shape=(n_timesteps, n_features), padding='same'))
conv1d_model.add(tf.keras.layers.Conv1D(filters=70, kernel_size=3, activation='relu', padding='same'))
conv1d_model.add(tf.keras.layers.Dropout(rate=0.5))
conv1d_model.add(tf.keras.layers.MaxPooling1D(pool_size=1))
conv1d_model.add(tf.keras.layers.Flatten())
conv1d_model.add(tf.keras.layers.Dense(150, activation='relu'))
conv1d_model.add(tf.keras.layers.Dense(n_outputs, activation='softmax'))

conv1d_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [53]:
conv1d_model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_22 (Conv1D)          (None, 128, 70)           1960      
                                                                 
 conv1d_23 (Conv1D)          (None, 128, 70)           14770     
                                                                 
 dropout_11 (Dropout)        (None, 128, 70)           0         
                                                                 
 max_pooling1d_11 (MaxPoolin  (None, 128, 70)          0         
 g1D)                                                            
                                                                 
 flatten_11 (Flatten)        (None, 8960)              0         
                                                                 
 dense_22 (Dense)            (None, 150)               1344150   
                                                     

In [54]:
training_history=conv1d_model.fit(X_train, y_train, epochs=50, batch_size=32)

Epoch 1/50
230/230 [==============================] - 3s 6ms/step - loss: 0.3666 - accuracy: 0.8526
Epoch 2/50
230/230 [==============================] - 1s 4ms/step - loss: 0.1384 - accuracy: 0.9442
Epoch 3/50
230/230 [==============================] - 1s 4ms/step - loss: 0.1131 - accuracy: 0.9516
Epoch 4/50
230/230 [==============================] - 1s 4ms/step - loss: 0.1215 - accuracy: 0.9491
Epoch 5/50
230/230 [==============================] - 1s 4ms/step - loss: 0.1258 - accuracy: 0.9476
Epoch 6/50
230/230 [==============================] - 1s 4ms/step - loss: 0.0983 - accuracy: 0.9573
Epoch 7/50
230/230 [==============================] - 1s 4ms/step - loss: 0.1135 - accuracy: 0.9535
Epoch 8/50
230/230 [==============================] - 1s 4ms/step - loss: 0.0840 - accuracy: 0.9612
Epoch 9/50
230/230 [==============================] - 1s 4ms/step - loss: 0.0774 - accuracy: 0.9638
Epoch 10/50
230/230 [==============================] - 1s 4ms/step - loss: 0.0757 - accuracy: 0.9650

In [55]:
_, accuracy=conv1d_model.evaluate(X_test, y_test, batch_size=32)

93/93 [==============================] - 1s 7ms/step - loss: 0.8416 - accuracy: 0.9209


In [56]:
accuracy

0.9209365248680115

In [57]:
def model_maker(X_train, y_train, X_test, y_test):
    n_timesteps, n_features, n_outputs=X_train.shape[1], X_train.shape[2], y_train.shape[1]
    
    conv1d_model=tf.keras.Sequential()
    conv1d_model.add(tf.keras.layers.Conv1D(filters=70, kernel_size=3, activation='relu', input_shape=(n_timesteps, n_features), padding='same'))
    conv1d_model.add(tf.keras.layers.Conv1D(filters=70, kernel_size=3, activation='relu', padding='same'))
    conv1d_model.add(tf.keras.layers.Dropout(rate=0.5))
    conv1d_model.add(tf.keras.layers.MaxPooling1D(pool_size=1))
    conv1d_model.add(tf.keras.layers.Flatten())
    conv1d_model.add(tf.keras.layers.Dense(150, activation='relu'))
    conv1d_model.add(tf.keras.layers.Dense(n_outputs, activation='softmax'))

    conv1d_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    training_history=conv1d_model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)
    _, accuracy=conv1d_model.evaluate(X_test, y_test, batch_size=32)
    return accuracy

In [58]:
def summarize_results(scores):
    print(scores)
    m, s = np.mean(scores), np.std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [59]:
def experiment(num_of_repeats, X_train, y_train, X_test, y_test):
    scores = list()
    for n in range(num_of_repeats):
        score = model_maker(X_train, y_train, X_test, y_test)
        score = score * 100.0
        print('>#%d: %.3f' % (n+1, score))
        scores.append(score)
    summarize_results(scores)

In [60]:
experiment(10, X_train, y_train, X_test, y_test)

93/93 [==============================] - 0s 2ms/step - loss: 1.1049 - accuracy: 0.9321
>#1: 93.213
93/93 [==============================] - 0s 3ms/step - loss: 1.5984 - accuracy: 0.9182
>#2: 91.822
93/93 [==============================] - 0s 2ms/step - loss: 0.8388 - accuracy: 0.9291
>#3: 92.908
93/93 [==============================] - 0s 3ms/step - loss: 0.8862 - accuracy: 0.9152
>#4: 91.517
93/93 [==============================] - 0s 3ms/step - loss: 1.0493 - accuracy: 0.9050
>#5: 90.499
93/93 [==============================] - 0s 3ms/step - loss: 0.7112 - accuracy: 0.9315
>#6: 93.146
93/93 [==============================] - 0s 3ms/step - loss: 1.6337 - accuracy: 0.9046
>#7: 90.465
93/93 [==============================] - 0s 3ms/step - loss: 0.8310 - accuracy: 0.9125
>#8: 91.245
93/93 [==============================] - 0s 3ms/step - loss: 0.7499 - accuracy: 0.9348
>#9: 93.485
93/93 [==============================] - 0s 3ms/step - loss: 0.6108 - accuracy: 0.9332
>#10: 93.315
[93.21343